<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/validation/football_TF_models_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [1]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [2]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Downloads

In [3]:
dataset_name = './dataset.npz'
data_version = 'data_221101/'

In [4]:
project = neptune.init_project(
    name="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset_npz'].download(dataset_name)
project[data_version + 'vectors'].download('./vectors.npz')
project[data_version + 'word2vec'].download('./word2vec_vs16_sg1.wordvectors.tar.gz')
word2vec_params = project[data_version+ 'word2vec_params'].fetch()
project[data_version + 'vectors'].download('./vectors.npz')
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


### Imports

In [5]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

#import dask.dataframe as dd
import subprocess
from glob import glob
from tqdm import tqdm

1.3.5
1.21.6


In [6]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

In [7]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from itertools import cycle

In [8]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from keras.layers import Embedding, SimpleRNN, Dense, Bidirectional, GRU, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.data import Dataset
#from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.models import Sequential

### Code

#####Functions

In [9]:
def run_bash(bashCommand:str, nameCommand = ''):
        process = subprocess.Popen([bashCommand], 
                           shell=True)
        _, error = process.communicate()
        if error:
            print(f'{nameCommand} error:\n', error)

In [10]:
def get_fp_tp(y, predicted, threshold):
    tn, fp, fn, tp = confusion_matrix(y, (predicted > threshold).astype(int)).ravel()
    return tn, fp, fn, tp

In [11]:
def get_roc_curve_data(y, y_pred):
    # defining fpr and tpr
    tpr = []
    fpr = []
    # определяем положительные и отрицательные
    positives = np.sum(y == 1)
    negatives = np.sum(y == 0)
    # перебираем пороговые значения и получаем количество ложно и истинно положительных результатов 
    for th in np.linspace(0,1,100):
        tn, fp, fn, tp = get_fp_tp(y, y_pred, th)
        tpr.append(tp/positives)
        fpr.append(fp/negatives)
    return tpr, fpr

In [12]:
def plot_roc_auc(y_true, x_predicted, data_split = 'train', eval_train = 0, eval_test = 0):
    tpr, fpr = get_roc_curve_data(y_true, x_predicted)
    fig = px.area(
        x = fpr, y = tpr,
        title=f'ROC Curve {data_split}<br>accuracy train: {round(eval_train * 100, 2)} %<br>accuracy test: {round(eval_test * 100, 2)} %',
        labels=dict(x='False Positive Rate', y='True Positive Rate')
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    fig.update_layout(
        height = 600,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    fig.show()
    return fig

In [13]:
def get_precision_recall_curve(y, y_pred):
    precision = []
    recall = []
    for th in np.linspace(0,1,100):
        tn, fp, fn, tp = get_fp_tp(y, y_pred, th)
        if ((tp + fn) != 0) & ((tp + fp) != 0):
            precision.append(tp/(tp + fp))
            recall.append(tp/(tp + fn))
    return precision, recall

In [14]:
def plot_precision_recall(y_true, x_predicted, data_split = 'train'):
    precision, recall = get_precision_recall_curve(y_true, x_predicted)
    fig = px.area(
        x = recall, y = precision,
        title=f'Precision Recall Curve {data_split}',
        labels=dict(x='Recall', y='Presion')
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    fig.update_layout(
        height = 600,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    fig.show()
    return fig

In [15]:
def get_f1_curve(y, y_pred):
    threshold = []
    f1 = []
    for th in np.linspace(0,1,100):
        threshold.append(th)
        f1.append(f1_score(y, (y_pred > th).astype(int)))
    return threshold, f1

In [16]:
def plot_f1(y_true, x_predicted, data_split = 'train'):
    threshold, f1 = get_f1_curve(y_true, x_predicted)
    fig = px.area(
        x = threshold, y = f1,
        title=f'F1 Curve {data_split}',
        labels=dict(x='threshold', y='F1'),
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    fig.update_layout(
        width = 600,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    fig.show()
    return fig

In [17]:
def plot_confusion_matrix(cfm, data_split = 'train', threshold = ''):
    title_text = 'confusion matrix ' + data_split
    if threshold != '':
        title_text = title_text +'<br>' + f'threshold = {threshold}'
    x = ['Away', 'Home']
    y = ['Away', 'Home']
    fig = px.imshow(cfm, x=x, y=y, color_continuous_scale='Purples', text_auto=True)
    fig.update_xaxes(title_text = 'Predicted Label')
    fig.update_yaxes(title_text = 'True Label')
    fig.update_layout(
        height = 400,
        title_text = title_text,
        title_font_size=20,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    fig.update_coloraxes(showscale=False)
    fig.show()
    return fig

In [18]:
def get_profit_curve(y, y_pred, Line_production, bet_type = 'fixed', strategy = 'simple', reverse_bet = False):
    #fixed, divk, divk-1
    #simple, complex
    threshold = []
    profit = []
    bet_qty_list = []
    if bet_type == 'divk':
        profit_size = (Line_production - 1) / Line_production
        bet_size = Line_production
    elif bet_type == 'divk-1':
        profit_size = (Line_production - 1) / (Line_production - 1)
        bet_size = Line_production - 1
    else:
        profit_size = (Line_production - 1)
        bet_size = Line_production /Line_production
    for th in np.linspace(0,1,1001):
        threshold.append(th)
        if reverse_bet:
            if strategy == 'simple':
                vector_th = y_pred[:,0] < th
            if strategy == 'complex':
                vector_th = (y_pred[:,0] * Line_production / 10) < th
            bet_qty_list.append(vector_th.sum())
            profit.append(((-1) * (y[vector_th] - 1) * profit_size[vector_th] + ((-1) * y[vector_th]) / bet_size[vector_th]).sum())
        else:            
            if strategy == 'simple':
                vector_th = y_pred[:,0] > th
            if strategy == 'complex':
                vector_th = (y_pred[:,0] * Line_production / 10) > th
            bet_qty_list.append(vector_th.sum())
            profit.append((y[vector_th] * profit_size[vector_th] + (y[vector_th] - 1) / bet_size[vector_th]).sum())

    return threshold, profit, bet_qty_list

In [19]:
def get_profit_validation(y_true, x_predicted, Line_production, reverse_bet = False):
    scatters_dicts = dict(
        scatter1 = dict(x = [], y = [], name = '', fill = 'tozeroy', yaxis = '', xaxis = ''),
        scatter2 = dict(x = [], y = [], name = '', line = dict(color='rgb(33,113,181)', dash='dash'), yaxis = '', xaxis = ''),
        scatter3 = dict(x = [], y = [], name = '', line = dict(color='rgb(107,174,214)', dash='dash'), yaxis = '', xaxis = '')
                    )
    qty_color = 'blue'
    prof_qty_color = 'rgb(8,48,107)'
    bet_type_list = ['fixed', 'divk', 'divk-1']
    bet_size_list = ['1', '1/K', '1/(K-1)']
    strategy_list = ['simple', 'complex']
    strategy_name_list = ['threshold', 'pred*k']
    domain_list = [[0.55, 1], [0., 0.5]]
    layout_dict = {}
    data_list = []
    title_text = f'Profit & bet qty for validation model in <a href="http://neptune.ai">neptune.ai</a>:     FOOT-{target_result}-{model_num}'
    for cnt_str, strategy in enumerate(strategy_list):
        for cnt_bet, bet_type in enumerate(bet_type_list):
            y_anchor = str((9 * cnt_str) + (cnt_bet*3) + 1)
            threshold, profit, bet_qty_list = get_profit_curve(
                y_true, x_predicted, Line_production,
                bet_type = bet_type, strategy = strategy, reverse_bet = reverse_bet)
            xaxis_num = str((3 * cnt_str) + (cnt_bet + 1))
            layout_dict.update({
                    'xaxis' + xaxis_num:{
                        'domain':[0.35 * cnt_bet, 0.35 * cnt_bet + 0.3],
                        'title':f'bet={bet_size_list[cnt_bet]}, strategy:{strategy_name_list[cnt_str]}',
                        'anchor':'y' + y_anchor}
                                })
            for cnt_scatter in range(1, 4):
                yaxis_num = str((9 * cnt_str) + (cnt_bet*3) + cnt_scatter)
                scatter_num = 'scatter' + str(cnt_scatter)
                scatters_dicts[scatter_num]['x'] = threshold
                if cnt_scatter == 1:
                    scatters_dicts[scatter_num]['y'] = profit
                    scatters_dicts[scatter_num]['name'] = 'profit_' +xaxis_num
                    scatters_dicts[scatter_num]['xaxis'] = 'x' + xaxis_num
                    scatters_dicts[scatter_num]['yaxis'] = 'y' + yaxis_num
                    layout_dict.update({
                        'yaxis' + yaxis_num:{
                            'domain':domain_list[cnt_str],
                            'title':'', #'Profit',
                            'range':[-10,int(max(profit) * 1.1)],
                            'anchor':'x' + xaxis_num
                    }})
                elif cnt_scatter == 2:
                    scatters_dicts[scatter_num]['y'] = np.array(profit) / np.array(bet_qty_list)
                    scatters_dicts[scatter_num]['name'] = 'profit_' + xaxis_num
                    scatters_dicts[scatter_num]['xaxis'] = 'x' + xaxis_num
                    scatters_dicts[scatter_num]['yaxis'] = 'y' + yaxis_num
                    layout_dict.update({
                        'yaxis' + yaxis_num:{
                            'domain':domain_list[cnt_str],
                            'title':'', 'zeroline':True,
                            'side':'right', 'anchor':'x' + xaxis_num,
                            'overlaying':'y' + y_anchor}})
                elif cnt_scatter == 3:
                    scatters_dicts[scatter_num]['y'] = np.array(bet_qty_list) / 1000
                    scatters_dicts[scatter_num]['name'] = 'profit_' + xaxis_num
                    scatters_dicts[scatter_num]['xaxis'] = 'x' + xaxis_num
                    scatters_dicts[scatter_num]['yaxis'] = 'y' + yaxis_num
                    layout_dict.update({
                        'yaxis' + yaxis_num:{
                            'domain':domain_list[cnt_str],
                            'visible':False, 'showgrid':True,
                            'side':'right', 'anchor':'x' + xaxis_num,
                            'overlaying':'y' + y_anchor}})
                data_list += [go.Scatter(**scatters_dicts['scatter' + str(cnt_scatter)])]
    layout_dict.update({
        'width':1400,
        'height':800,
        'title_x':0.5,
        'title_text':title_text,
        'paper_bgcolor':'rgb(229, 237, 247)',
        'plot_bgcolor':'rgb(229, 237, 247)',
        'showlegend':False 
                        })
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout)

In [20]:
def plot_equity(y_true, y_pred, Line_production, th, bet_type = 'fixed', strategy = 'simple', data_split = 'validation', reverse_bet = False):\
    #fixed, divk, divk-1
    #simple, complex
    if bet_type == 'divk':
        profit_size = (Line_production - 1) / Line_production
        bet_size = Line_production
    elif bet_type == 'divk-1':
        profit_size = (Line_production - 1) / (Line_production - 1)
        bet_size = Line_production - 1
    else:
        profit_size = (Line_production - 1)
        bet_size = Line_production /Line_production
    
    if strategy == 'simple':
        vector_th = y_pred[:,0] > th
    if strategy == 'complex':
        vector_th = (y_pred[:,0] * Line_production / 10) > th
    mean_bet =np.mean(1/bet_size[vector_th])
    y = (y_true[vector_th] * profit_size[vector_th] + (y_true[vector_th] - 1) / bet_size[vector_th]).cumsum()
    bet_qty = vector_th.sum()
    title_text = f'Equity Curve {data_split} | threshold={th} | bet_type:{bet_type} | strategy:{strategy}' + \
                f'<br>bet_mean:{round(mean_bet, 4)} | ROI:{round(y[-1] * 100 / np.sum(1/bet_size[vector_th]), 2)}%' + \
                f' | Bet quantity:{bet_qty} | bet per day~{round(bet_qty / 270, 0)}<br>' + \
                f'Model in <a href="http://neptune.ai">neptune.ai</a>: FOOT-{target_result}-{model_num}'
    layout_dict = {}
    #fig = go.Figure()
    trace_equity = go.Scatter(y = y, fill='tozeroy', xaxis = 'x1', yaxis = 'y1') #'toself'
    layout_dict.update({'xaxis1':{'anchor':'y1'}, 'yaxis1':{'domain':[0.5, 1]}})
    max_profit = [y[0]] + [max(y[:x]) if max(y[:x]) > y[x] else y[x] for x in range(1, len(y_pred[vector_th]))]
    trace_drawdown =  go.Scatter(y = (y - max_profit) / mean_bet, fill='tozeroy', xaxis = 'x2', yaxis = 'y2')
    layout_dict.update({'xaxis2':{'anchor':'y2'}, 'yaxis2':{'domain':[0.25, 0.45], 'title':'drawdown in<br>mean(bet)'}})

    cnt = 0
    dd = []
    for var1 in (y - max_profit):
        if var1 < 0:
            cnt += 1
        else:
            cnt = 0
        dd.append(cnt)
    trace_time_dd = go.Bar(y = dd, xaxis = 'x3', yaxis = 'y3')
    layout_dict.update({'xaxis3':{'anchor':'y3', 'title':'time from last max in bets'}, 'yaxis3':{'domain':[0., 0.2]}})

    layout_dict.update({
    'width':1400,
    'height':800,
    'title_x':0.5,
    'title_text':title_text,
    'paper_bgcolor':'rgb(229, 237, 247)',
    'plot_bgcolor':'rgb(229, 237, 247)',
    'showlegend':False 
                    })
    data_list = [trace_equity, trace_drawdown, trace_time_dd]
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout)

#### Load tensors

In [21]:
dataset_name = './dataset.npz'

In [22]:
#@title Target
target_result = "HOME" #@param ["HOME", "DRAW", "AWAY"]
target_dict = {'HOME':0, 'DRAW':1, 'AWAY':2}
clmn = target_dict[target_result]
data_npz = np.load(dataset_name)
X_train, X_test, X_validation, X_production = \
data_npz['X_train'], data_npz['X_test'], data_npz['X_validation'], data_npz['X_production']
y_train, y_test, y_validation,  y_production = \
data_npz['y_class_train'][:, clmn], data_npz['y_class_test'][:, clmn], data_npz['y_class_validation'][:, clmn], data_npz['y_class_production'][:, clmn]
Line_production, embedding_matrix = data_npz['Line_production'], data_npz['embedding_matrix']
vectors_npz = np.load('./vectors.npz')
hasprem_vector, tier2_vector, tier3_vector = \
vectors_npz['hasprem_vector'], vectors_npz['tier2_vector'], vectors_npz['tier3_vector']

In [24]:
model_num = 1 # Указываем номер модели
neptune_model = f'FOOT-{target_result}221101'
neptune_model_version = neptune_model + f'-{model_num}'
model_version_params = dict(
    project = 'scomesse/football',
    model = neptune_model,
    api_token = api_key,
    with_id = neptune_model_version
)
PATH_TO_MODEL = './model.tar.gz'
model_version = neptune.init_model_version(**model_version_params)
model_version['model'].download(PATH_TO_MODEL)
RNN_params = model_version['parameters'].fetch()
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1/metadata


In [25]:
bashCommand = f"""
tar -zxvf {PATH_TO_MODEL}
"""
run_bash(bashCommand, 'tar_model')

In [26]:
model = load_model('./models_weights')

In [27]:
evaluate_train = model.evaluate(X_train, y_train)
evaluate_test = model.evaluate(X_test, y_test)
evaluate_validation = model.evaluate(X_validation, y_validation)
evaluate_production = model.evaluate(X_production, y_production)

1749/1749 [==============================] - 3s 1ms/step - loss: 0.6556 - acc: 0.6081


In [28]:
%%time
predict_train = model.predict(X_train) #.round().astype(int)
predict_test = model.predict(X_test) #.round().astype(int)
predict_production = model.predict(X_production) #.round().astype(int)

1749/1749 [==============================] - 2s 1ms/step
CPU times: user 2min 7s, sys: 5.74 s, total: 2min 12s
Wall time: 3min 8s


In [ ]:
model_version = neptune.init_model_version(**model_version_params)
model_version['train_accuracy'] = evaluate_train[1]
model_version['test_accuracy'] = evaluate_test[1]
model_version['production_accuracy'] = evaluate_production[1]
model_version['train_loss'] = evaluate_train[0]
model_version['test_loss'] = evaluate_test[0]
model_version['production_loss'] = evaluate_production[0]
model_version.stop()

In [29]:
fig = plot_roc_auc(
    y_train,
    predict_train,
    data_split = 'train',
    eval_train = evaluate_train[1],
    eval_test = evaluate_test[1]
    )
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_roc_auc_train'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1/metadata


In [30]:
fig = plot_roc_auc(
    y_test,
    predict_test,
    data_split = 'test',
    eval_train = evaluate_train[1],
    eval_test = evaluate_test[1]
    )
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_roc_auc_test'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [31]:
fig = plot_precision_recall(
    y_train,
    predict_train,
    data_split = 'train'
)
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_precision_recall_train'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [32]:
fig = plot_precision_recall(
    y_test,
    predict_test,
    data_split = 'test'
)
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_precision_recall_test'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [33]:
fig = plot_precision_recall(
    y_production,
    predict_production,
    data_split = 'production'
)
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_precision_recall_production'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [34]:
fig = plot_f1(
    y_train,
    predict_train,
    data_split = 'train'
)
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_f1_train'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [35]:
fig = plot_f1(
    y_test,
    predict_test,
    data_split = 'test'
)
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_f1_test'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [36]:
fig = plot_f1(
    y_production,
    predict_production,
    data_split = 'production'
)
model_version = neptune.init_model_version(**model_version_params)
model_version['plot_f1_production'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [37]:
thres = 0.44
f1_train = f1_score(y_train, (predict_train > thres).round().astype(int))
f1_test = f1_score(y_test,(predict_test > thres).round().astype(int))
f1_production = f1_score(y_production,(predict_production > thres).round().astype(int))
print(f1_train, f1_test, f1_production)

0.6103463611351901 0.6090962244956588 0.5945161910420634


In [38]:
cfm_train = confusion_matrix(y_train, (predict_train > thres).round().astype(int))
cfm_test = confusion_matrix(y_test, (predict_test > thres).round().astype(int))
cfm_production = confusion_matrix(y_production, (predict_production > thres).round().astype(int))
fig = plot_confusion_matrix(cfm_train, data_split = 'train', threshold = thres)

In [ ]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'plot_confusion_train_{thres}'].upload(neptune.types.File.as_html(fig))
model_version.stop()

In [39]:
fig = plot_confusion_matrix(cfm_test, data_split = 'test', threshold = thres)
model_version = neptune.init_model_version(**model_version_params)
model_version[f'plot_confusion_test_{thres}'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1/metadata


In [40]:
fig = plot_confusion_matrix(cfm_production, data_split = 'production', threshold = thres)
model_version = neptune.init_model_version(**model_version_params)
model_version[f'plot_confusion_production_{thres}'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221101/v/FOOT-HOME221101-1/metadata


In [41]:
fig = get_profit_validation(y_production, predict_production, Line_production[:,clmn])
fig.show()
fig.write_html(f'FOOT-{target_result}-{model_num}_profit_report.html')

<ipython-input-19-ca72c28a487e>:47: RuntimeWarning:

invalid value encountered in true_divide



In [ ]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'profit_validation'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-6
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-6/metadata


In [48]:
threshold = 0.208
bet_type = 'divk'
strategy = 'complex'
fig = plot_equity(y_production, 
                  predict_production, 
                  Line_production[:,clmn], 
                  threshold, bet_type = bet_type, strategy = strategy)
fig.show()

In [ ]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'equity_{bet_type}_{strategy}_th_{threshold}'].upload(neptune.types.File.as_html(fig))
model_version.stop()
fig.write_html(f'FOOT-{target_result}-{model_num}_{bet_type}_{strategy}_th_{threshold}_equity_report.html')

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-5
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-5/metadata


In [49]:
fig = get_profit_validation(y_production[hasprem_vector], predict_production[hasprem_vector], Line_production[:,clmn][hasprem_vector], reverse_bet = False)
fig.show()
fig.write_html(f'FOOT-{target_result}-{model_num}_hasprem_profit_report.html')

<ipython-input-19-ca72c28a487e>:47: RuntimeWarning:

invalid value encountered in true_divide



In [ ]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'profit_validation_hasprem'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-6
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-6/metadata


In [50]:
threshold = 0.125
bet_type = 'divk'
strategy = 'complex'
fig = plot_equity(y_production[hasprem_vector], 
                  predict_production[hasprem_vector], 
                  Line_production[:,clmn][hasprem_vector], 
                  threshold, bet_type = bet_type, strategy = strategy)
fig.show()

In [ ]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'equity_{bet_type}_{strategy}_th_{threshold}'].upload(neptune.types.File.as_html(fig))
model_version.stop()
fig.write_html(f'FOOT-{target_result}-{model_num}_{bet_type}_{strategy}_th_{threshold}_equity_report_hasprem.html')

https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-6
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-HOME221212/v/FOOT-HOME221212-6/metadata
